In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # titan RTX

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Load configurations

In [3]:
import json

class Opt():
    def __init__(self):
        pass

opt = Opt()
with open('manifest/FineTunedConvNet_MulticamFD@r2plus1d_18_cv.cfg', 'r') as f: 
    opt.__dict__ = json.load(f)

In [4]:
import utils.engine
from datasets import get_data_loader
from models import generate_network

import torch
import torch.optim as optim
import torch.nn as nn
import pytorch_warmup as warmup  # for LR warmup
import os
import numpy as np
import sklearn.metrics

In [ ]:
def train_one_fold(fold, metrics=['f1-score', 'accuracy', 'ap', 'roc_auc']):
    
    # Load data
    train_loader, test_loader, target_transform, n_outputs = get_data_loader(
            opt, fold=fold)
    
    # Init model
    model = generate_network(opt, n_outputs=n_outputs, target_transform=target_transform)
    
    # Define optimizer & schedulers
    params = [p for p in model.parameters() if p.requires_grad]
    warmup_period = len(train_loader) * 3

    torch.nn.utils.clip_grad_norm_(params, opt.max_gradnorm)

    optimizer = optim.AdamW(params, lr=opt.learning_rate, weight_decay=opt.weight_decay)
    lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 10], gamma=0.1)
    warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period=warmup_period)
    warmup_scheduler.last_step = -1
    
    # Define NeuralNetwork Wrapper Class
    net = utils.engine.VideoClassifier(model, optimizer, 
                                   n_folds=opt.n_folds,
                                   lr_scheduler=lr_scheduler, warmup_scheduler=warmup_scheduler)
    
    net.train(train_loader, test_loader, 
              n_epochs=opt.n_iter, validation_freq=20,
              multiple_clip=True, metrics=metrics, 
              save_dir=os.path.join(opt.ckpt_dir, opt.model_indicator))

# LOO(Leave-One-Out) cross validation

In [ ]:
for fold in range(1,opt.n_folds+1):
    # LOO cross-validation loop
    train_one_fold(fold, metrics={
        'f1-score': (lambda y_true, y_pred: sklearn.metrics.accuracy_score(y_true, y_pred), False),
        'roc_auc': (lambda y_true, y_score: sklearn.metrics.roc_auc_score(y_true, y_score), True),
        'ap': (lambda y_true, y_score: sklearn.metrics.average_precision_score(y_true, y_score), True),
        'f1-score': (lambda y_true, y_pred: sklearn.metrics.f1_score(y_true, y_pred), False),
        # custom callbacks
        'sensitivity': (lambda y_true, y_pred: np.logical_and(y_true==1, y_pred == 1).sum() / (y_true==1).sum(), False),
        'specificity': (lambda y_true, y_pred: np.logical_and(y_true==0, y_pred == 0).sum() / (y_true==0).sum(), False)
    })

In [5]:
import tqdm

res = []
for fold in tqdm.tqdm(range(1, opt.n_folds+1)):
    # Load data
    train_loader, test_loader, target_transform, n_outputs = get_data_loader(
            opt, fold=fold)

    # Init model
    model = generate_network(opt, n_outputs=n_outputs, target_transform=target_transform)

    # Define NeuralNetwork Wrapper Class
    net = utils.engine.VideoClassifier(model, optimizer=None)
    
    mpath = os.path.join(opt.ckpt_dir, opt.model_indicator) + '/' + 'model_fold-{}.pth'.format(fold)
    
    res.append(net.test(test_loader, metrics=['f1-score', 'accuracy', 'ap', 'roc_auc'], multiple_clip=True, 
             pretrained_path=mpath))

class-balancing...: 100%|██████████| 54/54 [01:10<00:00,  1.31s/it]

train : 856, test : 56

Load pretrained model from /data/GaitData/ckpt_dir/FineTunedConvNet_MulticamFD@r2plus1d_18/model_fold-1.pth...
Start Evaluation...


class-balancing...: 100%|██████████| 53/53 [01:11<00:00,  1.34s/it]

train : 840, test : 72

Load pretrained model from /data/GaitData/ckpt_dir/FineTunedConvNet_MulticamFD@r2plus1d_18/model_fold-2.pth...
Start Evaluation...


class-balancing...: 100%|██████████| 54/54 [01:09<00:00,  1.21s/it]

...

class-balancing...: 100%|██████████| 56/56 [01:14<00:00,  1.34s/it]

train : 888, test : 24

Load pretrained model from /data/GaitData/ckpt_dir/FineTunedConvNet_MulticamFD@r2plus1d_18/model_fold-21.pth...
Start Evaluation...


class-balancing...: 100%|██████████| 54/54 [01:22<00:00,  1.53s/it]

train : 864, test : 48

Load pretrained model from /data/GaitData/ckpt_dir/FineTunedConvNet_MulticamFD@r2plus1d_18/model_fold-22.pth...
Start Evaluation...


100%|██████████| 22/22 [33:22<00:00, 92.57s/it]


In [25]:
import pandas as pd
pd.DataFrame([ res[i][1]['video'] for i in range(opt.n_folds) ] ).mean()

video_f1-score    0.955187
video_accuracy    0.982008
video_ap          0.962618
video_roc_auc     0.989123
dtype: float64

In [26]:
import pandas as pd
pd.DataFrame([ res[i][1]['clip'] for i in range(opt.n_folds) ] ).mean()

clip_f1-score    0.659726
clip_accuracy    0.911503
clip_ap          0.804851
clip_roc_auc     0.919610
dtype: float64

In [28]:
model = generate_network(opt, n_outputs=n_outputs, target_transform=target_transform)
model

DataParallel(
  (module): FineTunedConvNet(
    (backbone): Sequential(
      (0): R2Plus1dStem(
        (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): BasicBlock(
          (conv1): Sequential(
            (0): Conv2Plus1D(
              (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
              (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
              (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0

In [ ]:
ls /data/GaitData/ckpt_dir/DefaultAGNet_MulticamFD@r2plus1d_18/

In [ ]:
# sklearn.metrics.classification_report(out_history_cat.argmax(1).detach().cpu().numpy(), target_history_cat.detach().cpu().numpy())

In [ ]:
import os
os.path.basename(os.path.join(opt.ckpt_dir, opt.model_indicator))

In [ ]:
import torch
y_true = torch.tensor([0, 0, 0,0, 0])
y_pred = torch.tensor([1, 1, 1,0, 0])

In [ ]:
# sensitivity
np.logical_and(y_true==1, y_pred == 1).sum() / (y_true==1).sum()

In [ ]:
# specificity
np.logical_and(y_true==0, y_pred == 0).sum() / (y_true==0).sum()

In [ ]:
print(sklearn.metrics.classification_report(y_true, y_pred))

In [ ]:
len(y_true.unique())